In [4]:
# Library imports

import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import sys

# --- Configuration ---
CRYPTO_TICKER = "BCH-USD"
TABLE_NAME = "bch_price"

# --- Database Connection Setup ---
db_user = 'bigmachinestudios'
db_password = 'MyDumbProject'
db_host = 'localhost' # e.g., 'localhost' or an IP address
db_port = '5432' # Default port for PostgreSQL
db_name = 'finance'

# Connection string for PostgreSQL.
db_connection_str = f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

# Create a SQLAlchemy engine
try:
    engine = create_engine(db_connection_str)
    print(f"[{datetime.now()}] Database engine created successfully.")
except Exception as e:
    print(f"Failed to create database engine: {e}", file=sys.stderr)
    print("Check your PostgreSQL connection details and ensure the server is running.", file=sys.stderr)
    sys.exit(1)


def fetch_and_insert_data(ndata):
    """
    Fetches the current price and volume for Bitcoin Cash and
    inserts the data into a PostgreSQL table. This function performs one single insertion.
    """
    print(f"Fetching data for {CRYPTO_TICKER}...")
    print(ndata)
    try:
        # Get the crypto data for the specified ticker
        crypto = yf.Ticker(CRYPTO_TICKER)
        # Get the most recent market data available
        current_data = crypto.history(period="1d", interval="1m").tail(1)

        if not current_data.empty:
            # Extract the latest price and volume
            latest_price = current_data['Close'].iloc[0]
            latest_volume = current_data['Volume'].iloc[0]
            timestamp = datetime.now()

            print(f"  -> Fetched Price: ${latest_price:,.2f}, Volume: {latest_volume:,.0f}")

            # Create a pandas DataFrame with the new data
            new_data_df = pd.DataFrame({
                "Timestamp": [timestamp],
                "Ticker": [CRYPTO_TICKER],
                "Price": [latest_price],
                "Volume": [latest_volume]
            })

            # Insert the DataFrame into the SQL table
            new_data_df.to_sql(
                TABLE_NAME,
                con=engine,
                if_exists='append',
                index=False
            )

            print(f"Successfully inserted data into PostgreSQL table '{TABLE_NAME}'.")
            print(new_data_df)

        else:
            print(f"Could not retrieve current data for {CRYPTO_TICKER}.")

    except Exception as e:
        print(f"An error occurred: {e}", file=sys.stderr)
        sys.exit(1)

# --- Main Execution ---
if __name__ == "__main__":
    fetch_and_insert_data(5)
    print(f"[{datetime.now()}] Script finished one successful run.")
    


[2025-07-17 13:35:57.808752] Database engine created successfully.
Fetching data for BCH-USD...
5
  -> Fetched Price: $495.23, Volume: 0
Successfully inserted data into PostgreSQL table 'bch_price'.
                   Timestamp   Ticker       Price  Volume
0 2025-07-17 13:35:57.916553  BCH-USD  495.228394       0
[2025-07-17 13:35:57.980872] Script finished one successful run.


In [26]:
# This version takes the whole day's data in 1min increments and loads it into table bch_test

# Library imports

import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import sys

# --- Configuration ---
CRYPTO_TICKER = "BCH-USD"
TABLE_NAME = "bch_test"

# --- Database Connection Setup ---
db_user = 'bigmachinestudios'
db_password = 'MyDumbProject'
db_host = 'localhost' # e.g., 'localhost' or an IP address
db_port = '5432' # Default port for PostgreSQL
db_name = 'finance'

# Connection string for PostgreSQL.
db_connection_str = f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

# Create a SQLAlchemy engine
try:
    engine = create_engine(db_connection_str)
    print(f"[{datetime.now()}] Database engine created successfully.")
except Exception as e:
    print(f"Failed to create database engine: {e}", file=sys.stderr)
    print("Check your PostgreSQL connection details and ensure the server is running.", file=sys.stderr)
    sys.exit(1)


def fetch_and_insert_data(nperiod,ninterval):
    """
    Fetches the current price and volume for Bitcoin Cash and
    inserts the data into a PostgreSQL table. This function performs one single insertion.
    """
    print(f"Fetching data for {CRYPTO_TICKER}...")

    try:
        # Get the crypto data for the specified ticker
        crypto = yf.Ticker(CRYPTO_TICKER)
        # Get the most recent market data available
        current_data = crypto.history(period=nperiod, interval=ninterval)

        if not current_data.empty:
            # print(current_data)
            # Create a pandas DataFrame with the new data
            new_data_df = current_data

            # Insert the DataFrame into the SQL table
            new_data_df.to_sql(
                TABLE_NAME,
                con=engine,
                if_exists='replace',
                index=True
            )

            print(f"Successfully inserted data into PostgreSQL table '{TABLE_NAME}'.")
            # print(new_data_df)

        else:
            print(f"Could not retrieve current data for {CRYPTO_TICKER}.")

    except Exception as e:
        print(f"An error occurred: {e}", file=sys.stderr)
        sys.exit(1)

# --- Main Execution ---
if __name__ == "__main__":
    data_period = "1d"    # The period over which to collect the data
    data_interval = "15m"  # The interval between entries
    fetch_and_insert_data(data_period, data_interval)
    print(f"[{datetime.now()}] Script finished one successful run.")
    


[2025-07-18 02:34:30.419381] Database engine created successfully.
Fetching data for BCH-USD...
Successfully inserted data into PostgreSQL table 'bch_test'.
[2025-07-18 02:34:30.635053] Script finished one successful run.
